In [1]:
            # !pip install alpaca_trade_api

     |████████████████████████████████| 40 kB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 74 kB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 6.8 MB/s  eta 0:00:01


In [2]:
import alpaca_trade_api as api

In [ ]:
api = tradeapi.REST()


In [41]:
# import os
# files = glob.glob('/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble*')
# files.sort(key=os.path.getmtime)

# action_trade =  sorted(files,key=os.path.getmtime)[-1]

/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble_441.csv
/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble_504.csv
/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble_567.csv
/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble_630.csv
/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble_693.csv
/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble_756.csv
/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble_819.csv
/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble_882.csv
/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble_945.csv
/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble_1008.csv
/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble_1071.csv
/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_t

In [3]:
from datetime import datetime, timedelta
validation_window = 63
rebalance_window = 63
today = datetime.today().strftime('%Y-%m-%d')
start = datetime.today() - timedelta(days=validation_window + rebalance_window)
start = start.strftime('%Y-%m-%d')

df = YahooDownloader(start_date = start,
                     end_date = today,
                     ticker_list = config.SP_500_TICKER).fetch_data()

fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

'2021-06-19'

In [10]:
ensemble_agent = DRLEnsembleAgent(df=processed_full,
                 train_period=(train_start,train_end),
                 val_test_period=(val_test_start,val_test_end),
                 rebalance_window=rebalance_window, 
                 validation_window=validation_window, 
                 **env_kwargs)

last_state_ensemble = []

ppo_sharpe_list = []
ddpg_sharpe_list = []
a2c_sharpe_list = []

model_use = []
validation_start_date_list = []
validation_end_date_list = []
iteration_list = []

insample_turbulence = ensemble_agent.df[(ensemble_agent.df.date<ensemble_agent.train_period[1]) & (ensemble_agent.df.date>=ensemble_agent.train_period[0])]
insample_turbulence_threshold = np.quantile(insample_turbulence.turbulence.values, 0.85)

start = time.time()
# for i in range(self.rebalance_window + self.validation_window, len(self.unique_trade_date), self.rebalance_window):
validation_start_date = self.unique_trade_date[i - self.rebalance_window - self.validation_window]
validation_end_date = self.unique_trade_date[i - self.rebalance_window]

validation_start_date_list.append(validation_start_date)
validation_end_date_list.append(validation_end_date)
iteration_list.append(i)

print("============================================")
## initial state is empty
if i - self.rebalance_window - self.validation_window == 0:
    # inital state
    initial = True
else:
    # previous state
    initial = False

# Tuning trubulence index based on historical data
# Turbulence lookback window is one quarter (63 days)
end_date_index = self.df.index[self.df["date"] == self.unique_trade_date[i - self.rebalance_window - self.validation_window]].to_list()[-1]
start_date_index = end_date_index - 63 + 1

historical_turbulence = self.df.iloc[start_date_index:(end_date_index + 1), :]

historical_turbulence = historical_turbulence.drop_duplicates(subset=['date'])

historical_turbulence_mean = np.mean(historical_turbulence.turbulence.values)

print(historical_turbulence_mean)

if historical_turbulence_mean > insample_turbulence_threshold:
    # if the mean of the historical data is greater than the 90% quantile of insample turbulence data
    # then we assume that the current market is volatile,
    # therefore we set the 90% quantile of insample turbulence data as the turbulence threshold
    # meaning the current turbulence can't exceed the 90% quantile of insample turbulence data
    turbulence_threshold = insample_turbulence_threshold
else:
    # if the mean of the historical data is less than the 90% quantile of insample turbulence data
    # then we tune up the turbulence_threshold, meaning we lower the risk
    turbulence_threshold = np.quantile(insample_turbulence.turbulence.values, 1)
print("turbulence_threshold: ", turbulence_threshold)

############## Environment Setup starts ##############
## training env
train = data_split(self.df, start=self.train_period[0], end=self.unique_trade_date[i - self.rebalance_window - self.validation_window])
self.train_env = DummyVecEnv([lambda: StockTradingEnv(train,
                                                    self.stock_dim,
                                                    self.hmax,
                                                    self.initial_amount,
                                                    self.buy_cost_pct,
                                                    self.sell_cost_pct,
                                                    self.reward_scaling,
                                                    self.state_space,
                                                    self.action_space,
                                                    self.tech_indicator_list,
                                                    print_verbosity=self.print_verbosity)])

validation = data_split(self.df, start=self.unique_trade_date[i - self.rebalance_window - self.validation_window],
                        end=self.unique_trade_date[i - self.rebalance_window])
############## Environment Setup ends ##############

############## Training and Validation starts ##############
print("======Model training from: ", self.train_period[0], "to ",
      self.unique_trade_date[i - self.rebalance_window - self.validation_window])
# print("training: ",len(data_split(df, start=20090000, end=test.datadate.unique()[i-rebalance_window]) ))
# print("==============Model Training===========")
print("======A2C Training========")
model_a2c = self.get_model("a2c",self.train_env,policy="MlpPolicy",model_kwargs=A2C_model_kwargs)
model_a2c = self.train_model(model_a2c, "a2c", tb_log_name="a2c_{}".format(i), iter_num = i, total_timesteps=timesteps_dict['a2c']) #100_000

print("======A2C Validation from: ", validation_start_date, "to ",validation_end_date)
val_env_a2c = DummyVecEnv([lambda: StockTradingEnv(validation,
                                                    self.stock_dim,
                                                    self.hmax,
                                                    self.initial_amount,
                                                    self.buy_cost_pct,
                                                    self.sell_cost_pct,
                                                    self.reward_scaling,
                                                    self.state_space,
                                                    self.action_space,
                                                    self.tech_indicator_list,
                                                    turbulence_threshold=turbulence_threshold,
                                                    iteration=i,
                                                    model_name='A2C',
                                                    mode='validation',
                                                    print_verbosity=self.print_verbosity)])
val_obs_a2c = val_env_a2c.reset()
self.DRL_validation(model=model_a2c,test_data=validation,test_env=val_env_a2c,test_obs=val_obs_a2c)
sharpe_a2c = self.get_validation_sharpe(i,model_name="A2C")
print("A2C Sharpe Ratio: ", sharpe_a2c)

print("======PPO Training========")
model_ppo = self.get_model("ppo",self.train_env,policy="MlpPolicy",model_kwargs=PPO_model_kwargs)
model_ppo = self.train_model(model_ppo, "ppo", tb_log_name="ppo_{}".format(i), iter_num = i, total_timesteps=timesteps_dict['ppo']) #100_000
print("======PPO Validation from: ", validation_start_date, "to ",validation_end_date)
val_env_ppo = DummyVecEnv([lambda: StockTradingEnv(validation,
                                                    self.stock_dim,
                                                    self.hmax,
                                                    self.initial_amount,
                                                    self.buy_cost_pct,
                                                    self.sell_cost_pct,
                                                    self.reward_scaling,
                                                    self.state_space,
                                                    self.action_space,
                                                    self.tech_indicator_list,
                                                    turbulence_threshold=turbulence_threshold,
                                                    iteration=i,
                                                    model_name='PPO',
                                                    mode='validation',
                                                    print_verbosity=self.print_verbosity)])
val_obs_ppo = val_env_ppo.reset()
self.DRL_validation(model=model_ppo,test_data=validation,test_env=val_env_ppo,test_obs=val_obs_ppo)
sharpe_ppo = self.get_validation_sharpe(i,model_name="PPO")
print("PPO Sharpe Ratio: ", sharpe_ppo)

print("======DDPG Training========")
model_ddpg = self.get_model("ddpg",self.train_env,policy="MlpPolicy",model_kwargs=DDPG_model_kwargs)
model_ddpg = self.train_model(model_ddpg, "ddpg", tb_log_name="ddpg_{}".format(i), iter_num = i, total_timesteps=timesteps_dict['ddpg'])  #50_000
print("======DDPG Validation from: ", validation_start_date, "to ",validation_end_date)
val_env_ddpg = DummyVecEnv([lambda: StockTradingEnv(validation,
                                                    self.stock_dim,
                                                    self.hmax,
                                                    self.initial_amount,
                                                    self.buy_cost_pct,
                                                    self.sell_cost_pct,
                                                    self.reward_scaling,
                                                    self.state_space,
                                                    self.action_space,
                                                    self.tech_indicator_list,
                                                    turbulence_threshold=turbulence_threshold,
                                                    iteration=i,
                                                    model_name='DDPG',
                                                    mode='validation',
                                                    print_verbosity=self.print_verbosity)])
val_obs_ddpg = val_env_ddpg.reset()
self.DRL_validation(model=model_ddpg,test_data=validation,test_env=val_env_ddpg,test_obs=val_obs_ddpg)
sharpe_ddpg = self.get_validation_sharpe(i,model_name="DDPG")

ppo_sharpe_list.append(sharpe_ppo)
a2c_sharpe_list.append(sharpe_a2c)
ddpg_sharpe_list.append(sharpe_ddpg)

print("======Best Model Retraining from: ", self.train_period[0], "to ",
      self.unique_trade_date[i - self.rebalance_window])
# Environment setup for model retraining up to first trade date
train_full = data_split(self.df, start=self.train_period[0], end=self.unique_trade_date[i - self.rebalance_window])
self.train_full_env = DummyVecEnv([lambda: StockTradingEnv(train_full,
                                                    self.stock_dim,
                                                    self.hmax,
                                                    self.initial_amount,
                                                    self.buy_cost_pct,
                                                    self.sell_cost_pct,
                                                    self.reward_scaling,
                                                    self.state_space,
                                                    self.action_space,
                                                    self.tech_indicator_list,
                                                    print_verbosity=self.print_verbosity)])
# Model Selection based on sharpe ratio
if (sharpe_ppo >= sharpe_a2c) & (sharpe_ppo >= sharpe_ddpg):
    model_use.append('PPO')

    model_ensemble = self.get_model("ppo",self.train_full_env,policy="MlpPolicy",model_kwargs=PPO_model_kwargs)
    model_ensemble = self.train_model(model_ensemble, "ensemble", tb_log_name="ensemble_{}".format(i), iter_num = i, total_timesteps=timesteps_dict['ppo']) #100_000
elif (sharpe_a2c > sharpe_ppo) & (sharpe_a2c > sharpe_ddpg):
    model_use.append('A2C')

    model_ensemble = self.get_model("a2c",self.train_full_env,policy="MlpPolicy",model_kwargs=A2C_model_kwargs)
    model_ensemble = self.train_model(model_ensemble, "ensemble", tb_log_name="ensemble_{}".format(i), iter_num = i, total_timesteps=timesteps_dict['a2c']) #100_000
else:
    model_use.append('DDPG')

    model_ensemble = self.get_model("ddpg",self.train_full_env,policy="MlpPolicy",model_kwargs=DDPG_model_kwargs)
    model_ensemble = self.train_model(model_ensemble, "ensemble", tb_log_name="ensemble_{}".format(i), iter_num = i, total_timesteps=timesteps_dict['ddpg']) #50_000

############## Training and Validation ends ##############

############## Trading starts ##############
print("======Trading from: ", self.unique_trade_date[i - self.rebalance_window], "to ", self.unique_trade_date[i])
#print("Used Model: ", model_ensemble)
last_state_ensemble = self.DRL_prediction(model=model_ensemble, name="ensemble",
                                         last_state=last_state_ensemble, iter_num=i,
                                         turbulence_threshold = turbulence_threshold,
                                         initial=initial)
############## Trading ends ##############

In [34]:
for ticker in actions_trade.columns:
    if actions_trade.iloc[-1][ticker] > 0:
        api.submit_order(
            symbol=ticker,
            qty=actions_trade.iloc[-1][ticker],
            side='buy',
            type='market',
            time_in_force='gtc'
        )
    elif actions_trade.iloc[-1][ticker] < 0:
        api.submit_order(
            symbol=ticker,
            qty= actions_trade.iloc[-1][ticker],
            side='sell',
            type='market',
            time_in_force='gtc'
        )

100